In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)


# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
OpenAI response: content='"IntelliHome Innovations"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 21, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CrMpf0px3H1cJFzEHa0K2jwNK2mY8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5f95-6b04-71c0-b26c-d1a7c490608d-0' usage_metadata={'input_tokens': 21, 'output_tokens': 7, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Create the Agent with Short-Term Memory

### Start a Conversation (Memory in Action)

In [2]:
# Create in-memory checkpoint (short-term memory)
checkpointer = InMemorySaver()

# Create model
model = ChatOpenAI(model="gpt-4o-mini")

# Create agent with memory
agent = create_agent(
    model=model,
    tools=[],              # no tools for now
    checkpointer=checkpointer
)


#  Test Memory Recall

In [3]:
# Thread ID identifies the conversation
config = {"configurable": {"thread_id": "demo-thread"}}

# First message
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "Hi, my name is Alex."}]},
    config
)

print(response1["messages"][-1].content)


Hi Alex! How can I assist you today?


# Show Memory Is Session-Scoped

In [4]:
new_config = {"configurable": {"thread_id": "new-session"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is my name?"}]},
    new_config
)

print(response["messages"][-1].content)


I'm sorry, but I don't have access to information about you, including your name. How can I assist you today?


=====================================================
SHORT-TERM MEMORY — IMPORTANT NOTES (LangChain)
=====================================================

1. What is Short-Term Memory?
-----------------------------
Short-term memory stores *temporary conversation context*.
It exists only during the lifetime of a session (thread).

It is NOT permanent storage.

Examples:
- Last few user messages
- Assistant responses
- Temporary reasoning context

-----------------------------------------------------

2. What Short-Term Memory Is NOT
--------------------------------
❌ Not long-term memory  
❌ Not stored in database (unless you explicitly do it)  
❌ Not persistent across restarts  
❌ Not user history storage  

If the app restarts → memory is lost.

-----------------------------------------------------

3. How LangChain Handles Short-Term Memory
------------------------------------------
LangChain uses:
- AgentState
- Checkpointers (e.g., InMemorySaver)
- `user_id` + `thread_id` to group conversations

Example:
    user_id = "user_123"
    thread_id = "chat_1"

Same `user_id` + `thread_id` → same memory  
Different pair            → new memory

-----------------------------------------------------

4. InMemorySaver (Default Behavior)
----------------------------------
- Stores memory in RAM
- Fast and simple
- Ideal for:
  ✔ local testing
  ✔ demos
  ✔ short conversations

Not suitable for:
  ❌ production persistence
  ❌ multi-instance systems

-----------------------------------------------------

5. Common Pitfalls
------------------
❌ Storing too many messages → high token cost  
❌ Not trimming old messages  
❌ Treating short-term memory as a database  
❌ Forgetting to set `user_id` and `thread_id`  

-----------------------------------------------------

6. Best Practices
-----------------
✔ Keep only last N messages  
✔ Summarize older conversations  
✔ Use `user_id` + `thread_id` consistently  
✔ Combine with long-term memory for real apps  

-----------------------------------------------------

7. Recommended Setup (Production)
--------------------------------
Short-term memory  → InMemorySaver  
Long-term memory   → Postgres / Vector DB  
Summarization      → Middleware  
User identity      → `user_id` + `thread_id`  

-----------------------------------------------------

8. Golden Rule
--------------
Short-term memory helps the model *think better*,
not *remember forever*.

=====================================================

# User Based + Thread 

In [5]:
user_id = "user_123"
thread_id = f"{user_id}:thread_001"   # common pattern: make thread unique per user

config = {"configurable": {"thread_id": thread_id}}


In [6]:
# 4) First message (store user_id in state + messages in short-term memory)
r1 = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Hi, remember my favorite language is Python."}],
        "user_id": user_id,
    },
    config,
)
print(r1["messages"][-1].content)


That's great! Python is a versatile and powerful programming language. Do you have any specific projects or topics in Python that you'd like to discuss or explore?


In [7]:
# 5) Second message (same thread_id -> memory continues)
r2 = agent.invoke(
    {
        "messages": [{"role": "user", "content": "What is my favorite language?"}],
        "user_id": user_id,
    },
    config,
)
print(r2["messages"][-1].content)


Your favorite language is Python! If you have any questions or topics related to Python that you'd like to discuss, feel free to share!


In [8]:
print(r1["messages"][-1].content)


That's great! Python is a versatile and powerful programming language. Do you have any specific projects or topics in Python that you'd like to discuss or explore?
